[View in Colaboratory](https://colab.research.google.com/github/Haiga/LearningToRank/blob/master/codes/functionsInC/notebooks/learningToRank.ipynb)

In [35]:
%%cu

#include <stdio.h>
#include <stdlib.h>


int NUM_FEATURES = 64;
int NUM_TOTAL_DOCS = 10000;
int LENGHT_DOC_ID = 14;
/*
Funções básicas: ordenação, inserção ...
*/
void sort(float* vect, int tamanho, int ascending){
    for(int i=0; i<tamanho; i++){
        for(int j=i; j<tamanho; j++){
            if(ascending == 1){
                if(vect[i]>vect[j]){
                    float temp = vect[i];
                    vect[i] = vect[j];
                    vect[j] = temp;
                }
            }else{
                if(vect[i]<vect[j]){
                    float temp = vect[i];
                    vect[i] = vect[j];
                    vect[j] = temp;
                }
            }
        }
    }
}

/*
GPU functions basics
Ao realizar a chamada ao kernel da GPU, a execução em CPU continua.
Ao tentar acessar alguma variável na CPU pela GPU, ou vice-versa teremos um Runtime Error.
A invocação da função cudaDeviceSynchronize(), é utilizada quando precisamos saber quando 
a GPU terminou de executar a função que foi invocada.
Acessos indevidos a memória da GPU também lançará Runtime Errors
*/
inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
    if (err != cudaSuccess) {
        fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
    }
    return err;
}

void verifyError(){
    checkCudaErr(cudaDeviceSynchronize(), "Syncronization");
    checkCudaErr(cudaGetLastError(), "GPU");
}

/*
GPU funções de propósito geral, soma de vetores, multiplicação de vetores. Soma geral de vetores ...
*/

__global__ void sumGPU(float* vetor_total, float* vetor_subsomas, int tamanho_vetor_total, int tamanho_vetor_sub){
    int posicao = threadIdx.x + blockIdx.x*blockDim.x;
    int bloco = tamanho_vetor_total/tamanho_vetor_sub;
    if(posicao <tamanho_vetor_sub -1){
        vetor_subsomas[posicao] =0;
        for(int i=0; i<bloco; i++){
        vetor_subsomas[posicao] = vetor_subsomas[posicao] + vetor_total[bloco*posicao + i];   
        }
    }
    else if(posicao<tamanho_vetor_sub){
        vetor_subsomas[posicao] =0;
        for(int i=0; i<bloco; i++){
            if((bloco*posicao + i)<tamanho_vetor_total){
                vetor_subsomas[posicao] = vetor_subsomas[posicao] + vetor_total[bloco*posicao + i];  
            }
        }
    }  
}

__global__ void copyVect(float* vetor_destino, float* vetor_origem, int tamanho){
    int posicao = threadIdx.x + blockIdx.x*blockDim.x;
    
    if(posicao <tamanho){
        vetor_destino[posicao] = vetor_origem[posicao];
    }    
    
}

__global__ void setValue_1d(float* vetor, int valor, int tamanho_vetor){
    int posicao = threadIdx.x + blockIdx.x*blockDim.x;
    if(posicao<tamanho_vetor){
        vetor[posicao] = valor;
    }
}

float intern_sum(float* vetor, int tamanho){
    float total =0;
    for(int i=0; i<tamanho; i++){
        total = total + vetor[i];
    }
    return total;
}

float array_sum(float* vetor, int tamanho){
    
    int grupo = 10;
    int tamanho_vetor_sub_somas = ceil(tamanho/grupo);
    float* sub_somas;
    int t_old;
    if(tamanho_vetor_sub_somas>=100){
        
        cudaMallocManaged(&sub_somas, tamanho_vetor_sub_somas*sizeof(float));
       
        int iter =0;
        float* temp;
        if(ceil(0.1*tamanho_vetor_sub_somas)>=10){
            cudaMallocManaged(&temp, ceil(0.1*tamanho_vetor_sub_somas)*sizeof(float));
        }
        
        while(tamanho_vetor_sub_somas>=10){
            if(iter==0){
                sumGPU<<<ceil(tamanho_vetor_sub_somas/grupo),grupo>>>(vetor, sub_somas, tamanho, tamanho_vetor_sub_somas); 
                verifyError();
                t_old = tamanho_vetor_sub_somas;
                tamanho_vetor_sub_somas = ceil(0.1*tamanho_vetor_sub_somas);
            }
            else{
                sumGPU<<<ceil(tamanho_vetor_sub_somas/grupo),grupo>>>(sub_somas, temp, t_old, tamanho_vetor_sub_somas); 
                verifyError();
                copyVect<<<ceil(tamanho_vetor_sub_somas/grupo),grupo>>>(sub_somas, temp, tamanho_vetor_sub_somas);
                verifyError();
                t_old = tamanho_vetor_sub_somas;
                tamanho_vetor_sub_somas = ceil(0.1*tamanho_vetor_sub_somas);

            }
            iter++;
        }
        tamanho_vetor_sub_somas = t_old;
    }
    else{
        return intern_sum(vetor, tamanho);
    }
    return intern_sum(sub_somas, tamanho_vetor_sub_somas);
}

__global__ void multiplyVetor(float* vetorDestino, float* outroVetor, int tamanho){
    int posicao = threadIdx.x + blockIdx.x*blockDim.x;
    if(posicao<tamanho){
        vetorDestino[posicao] = vetorDestino[posicao]*outroVetor[posicao];
    }
}

/*
Funções para trabalhar com matrizes e arrays com ponteiros
*/
float* alocarArray(int tamanho)
{
  float* inicio;
  inicio = (float*)malloc(tamanho*sizeof(float));
	return inicio;
}

void setValue(float* arranjo,int tamanho,int value) {
	for (int i = 0; i < tamanho; i++) {
		arranjo[i] = value;
	}
}

float** alocarMatriz(int nLinha, int nColuna)
{
	float **matriz = (float**)malloc(nLinha * sizeof(float*));

	for (int i = 0; i < nLinha; i++) { 
		matriz[i] = alocarArray(nColuna);
	}
	return matriz; 
}

char* alocarString(int tamanho)
{
	char* inicio;
	inicio = (char*)malloc(tamanho * sizeof(char));
	return inicio;
}

char** alocarStrings(int nLinha, int nColuna)
{
	char **matriz = (char**)malloc(nLinha * sizeof(char*));

	for (int i = 0; i < nLinha; i++) {
		matriz[i] = alocarString(nColuna);
	}
	return matriz;
}

/*
Funções para tratamento de strings (vetores de char)
*/
float getValueFromWord(char word[]) {
	int initOfValue = 0;
	while (word[initOfValue] != ':') {
		initOfValue = initOfValue + 1;
	}
	initOfValue = initOfValue + 1;
	char textValue[8];
	for (int i = 0; i<8; i++) {
		textValue[i] = word[i + initOfValue];
	}
	return atof(textValue);
}
void copy(char word1[], char word2[], int tamanho) {
	for (int i = 0; i < tamanho; i++) {
		word1[i] = word2[i];
	}
	word1[tamanho] = '\0';
}

//Funções de corte dos dados
/*
Dado um conjunto de Dados (em forma matricial), retorna uma matriz com as features
especificadas no array features
*/
float** cutFeatures(float** data, int* features, int tamanho, int quantidadeDocs)
{

	float **matriz = alocarMatriz(quantidadeDocs, tamanho);
	for (int j = 0; j < tamanho; j++) {
		for (int contDoc = 0; contDoc < quantidadeDocs; contDoc++) {
			matriz[contDoc][j] = data[contDoc][features[j]];
		}
		
	}
	return matriz;
}
/*
Dado um conjunto de Dados (em forma matricial), retorna uma matriz com as consultas
especificadas no array docs
*/
float** cutDocs(float** data, int* docs, int tamanho, int quantidadeFeatures) {
	float **matriz = alocarMatriz(tamanho, quantidadeFeatures);
	for (int i = 0; i < tamanho; i++) {
		for (int contFeature = 0; contFeature < quantidadeFeatures; contFeature++) {
			matriz[i][contFeature] = data[docs[i]][contFeature];
		}

	}
	return matriz;
}
/*
Funções de medidas de relevência
*/
float* precisionInN(float* relevance, int tamanho) {
	float* pan = alocarArray(tamanho);
  cudaMallocManaged (&pan, tamanho);
	pan[0] = relevance[0]/1;
	for (int i = 1; i < tamanho; i++) {
		pan[i] = (pan[i - 1]*(i) + relevance[i])/(i+1);
	}
	return pan;
}

float map_IN_CPU(float* relevance, int tamanho) {
	float* map = precisionInN(relevance, tamanho);
	float sum = 0;
	float totalRelevantes = 0;
	for (int i = 0; i < tamanho; i++) {
		map[i] = map[i]*relevance[i];
	}
	for (int i = 0; i < tamanho; i++) {
		if (relevance[i] == 1) {
			totalRelevantes += 1;
		}
	}
	for (int i = 0; i < tamanho; i++) {
		sum = sum + map[i];
	}
	return sum / totalRelevantes;
}


float map(float* relevance, int tamanho) {
	float* map_arr = precisionInN(relevance, tamanho);
	float soma = 0;
	float totalRelevantes = 0;
	
    dim3 threads_per_block(32, 32, 1);
    dim3 number_of_blocks(tamanho / threads_per_block.x + 1, tamanho / threads_per_block.y + 1, 1);
    multiplyVetor <<<number_of_blocks , threads_per_block>>> (map_arr, relevance, tamanho);
    verifyError();
	for (int i = 0; i < tamanho; i++) {
		if (relevance[i] == 1) {
			totalRelevantes += 1;
		}
	}
	for (int i = 0; i < tamanho; i++) {
		soma = soma + map_arr[i];
	}
	return soma / totalRelevantes;
}

/*
Medidas de precisão: NDCG, CG, DCG, IDCG. Trabalhadas em CPU e GPU
*/
float* cg(float* relevance, int tamanho) {
	float* cg_arr = alocarArray(tamanho);
	cg_arr[0] = relevance[0];
	for (int i = 1; i < tamanho; i++) {
		cg_arr[i] = cg_arr[i - 1] + relevance[i];
	}
	return cg_arr;
}

float* vec_dcg(float* relevance, int tamanho) {
	float* dcg_arr = alocarArray(tamanho);
    dcg_arr[0] = relevance[0];
	for (int i = 1; i < tamanho; i++) {
		dcg_arr[i] = dcg_arr[i - 1] + relevance[i]/log2((float)(i+1));
	}
	return dcg_arr;
}

float* vec_idcg(float* relevance, int tamanho) {
	float* idcg_arr = alocarArray(tamanho);
	sort(relevance, tamanho, 0);
    idcg_arr[0] = relevance[0];
	for (int i = 1; i < tamanho; i++) {
		idcg_arr[i] = idcg_arr[i - 1] + relevance[i]/log2((float)(i+1));
	}
	return idcg_arr;
}

float* vec_ndcg(float* relevance, int tamanho){
    float* dcg_arr = vec_dcg(relevance, tamanho);
    float* idcg_arr = vec_idcg(relevance, tamanho);
    float* ndcg_arr = alocarArray(tamanho);
    for(int i=0; i<tamanho; i++){
        ndcg_arr[i] = dcg_arr[i]/idcg_arr[i];
    }
    return ndcg_arr;
}

float ndcg(float* relevance, int tamanho){
    float* dcg_arr = vec_dcg(relevance, tamanho);
    float* idcg_arr = vec_idcg(relevance, tamanho);
  
    if(idcg_arr[tamanho-1]!=0){
        return dcg_arr[tamanho-1]/idcg_arr[tamanho-1];          
    }
    else{
        return 0;
    }
    
}
__global__ void vec_dg(float* relevance, float* vetorResultante, int tamanho){
    int posicao = threadIdx.x + blockIdx.x*blockDim.x;
    if(posicao<tamanho){
        vetorResultante[posicao] = relevance[posicao]/log2((float)(posicao+1));
    }
}
float ndcg_GPU(float* relevance, int tamanho){
    float* vetorResultante;
    cudaMallocManaged(&vetorResultante, tamanho*sizeof(float));
    vec_dg<<<ceil(tamanho/32)+1,32>>>(relevance, vetorResultante, tamanho);
    verifyError();
    vetorResultante[0]= relevance[0];
    
    float dcg = array_sum(vetorResultante, tamanho);
    
    sort(relevance, tamanho, 0);
    vec_dg<<<ceil(tamanho/32)+1,32>>>(relevance, vetorResultante, tamanho);
    verifyError();
    vetorResultante[0]= relevance[0];
    float idcg = array_sum(vetorResultante, tamanho);
    
    if(idcg!=0){
        return dcg/idcg;
    }
    else{
        return 0;
    }
}

int main() {
	FILE *fp;
	char var[30];
	fp = fopen("drive/Colab Notebooks/data/2003_td_dataset/Fold1/Norm.test.txt", "r");
	
	int contWord = 0;
	int contDoc = 0;

	float** data = alocarMatriz(NUM_TOTAL_DOCS, NUM_FEATURES + 1);
	//float* relevancia = (float*)malloc(NUM_TOTAL_DOCS * sizeof(float));
    float* relevancia;
    cudaMallocManaged(&relevancia, NUM_TOTAL_DOCS*sizeof(float));  
	char** docIds = alocarStrings(NUM_TOTAL_DOCS, LENGHT_DOC_ID);
	if (fp != NULL)
	{
		while (fscanf(fp, "%s", var) != 0) {
			if (contWord == 0) {
				relevancia[contDoc] = atof(var);
			}
			else if (contWord <= (NUM_FEATURES + 1)) {
				data[contDoc][contWord-1] = getValueFromWord(var);
			}

			else if(contWord == (NUM_FEATURES + 4)) {
				copy(docIds[contDoc], var, LENGHT_DOC_ID);
				contWord = -1;
				contDoc = contDoc + 1;
				if (contDoc == NUM_TOTAL_DOCS) {
					break;
				}
			}

			contWord = contWord + 1;
		}
		fclose(fp);
	}
	else {
		printf("Cannot open file");
	}
  

    float* features_ndcg = alocarArray(NUM_FEATURES);
    //float* features = alocarArray(NUM_TOTAL_DOCS);
    float* features;
    cudaMallocManaged(&features, NUM_TOTAL_DOCS*sizeof(float));
    for(int i=0; i<NUM_FEATURES; i++){
        for(int j=0; j<NUM_TOTAL_DOCS; j++){
            features[j] = data[j][i+1];
        }
        features_ndcg[i] = ndcg_GPU(features, NUM_TOTAL_DOCS);
        //verifyError();
        //printf("%f --", features_ndcg[i]);
    }
    int* ordem = (int*)malloc(NUM_FEATURES*sizeof(int));
    for(int i=0; i<NUM_FEATURES;i++){
        ordem[i] = i+1;
    }
    for(int i=0; i<NUM_FEATURES; i++){
        for(int j=i; j<NUM_FEATURES; j++){
            if(features_ndcg[i]<features_ndcg[j]){
                float temp = features_ndcg[i];
                features_ndcg[i] = features_ndcg[j];
                features_ndcg[j] = temp;
                int temp2 = ordem[i];
                ordem[i] = ordem[j];
                ordem[j] = temp2;
            }    
        }
    }
    for(int i=0; i<NUM_FEATURES;i++){
        printf("%i - ", ordem[i]);
    }
    cudaFree(relevancia);
    free(data);
    free(docIds);
    //printf("%f ---- ", result);
    //printf("%f ---- ", result2);
	//printf(&var);

	return 0;
}
	


'21 - 26 - 36 - 40 - 52 - 30 - 31 - 58 - 35 - 19 - 41 - 42 - 59 - 18 - 25 - 44 - 47 - 46 - 3 - 13 - 48 - 22 - 23 - 1 - 57 - 27 - 20 - 16 - 11 - 5 - 37 - 28 - 15 - 38 - 50 - 43 - 45 - 17 - 12 - 2 - 32 - 24 - 4 - 33 - 14 - 61 - 62 - 64 - 39 - 29 - 56 - 55 - 51 - 49 - 63 - 54 - 53 - 60 - 34 - 10 - 6 - 7 - 8 - 9 - '

In [40]:
%%cu

#include <stdio.h>
#include <stdlib.h>
inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
  if (err != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
  }
  return err;
}

__global__ void multiplyVetor(float* vetorDestino, float* outroVetor, int tamanho){
    int posicao = threadIdx.x + blockIdx.x*blockDim.x;
    if(posicao<tamanho){
        vetorDestino[posicao] = vetorDestino[posicao]*outroVetor[posicao];
    }
}
__global__ void setValue(float* vetor, int tamanho){
    int posicao = threadIdx.x + blockIdx.x*blockDim.x;
    if(posicao<tamanho){
        vetor[posicao] = posicao;
    }
}
__global__ void dcg(float* vetor, int tamanho){
    int posicao = threadIdx.x + blockIdx.x*blockDim.x;
    if(posicao<tamanho && posicao!=0){
        vetor[posicao] = vetor[posicao-1]+ vetor[posicao];
    }
}
float* alocarArray(int tamanho)
{
	float* inicio;
	inicio = (float*)malloc(tamanho*sizeof(float));
	return inicio;
}
int main(){

    float* v1;
    float* v2;
    cudaMallocManaged(&v1, 10*sizeof(float));
    cudaMallocManaged(&v2, 10*sizeof(float));

    float* v3 = alocarArray(10);
    for(int i=0; i<10; i++){
        v3[i] = i;
    }
    float* v9;
    cudaMallocManaged(&v9, 100000*sizeof(float));
    setValue<<<512,256>>>(v9,100000);
    float* v10;
    cudaMallocManaged(&v10, 100000*sizeof(float));
    setValue<<<512,256>>>(v10,100000);

    dcg<<<512,256>>>(v10, 100000);
    setValue<<<32,32>>>(v1,10);
    setValue<<<32,32>>>(v2,10);

    multiplyVetor<<<32,32>>>(v1, v2, 10);
    checkCudaErr(cudaDeviceSynchronize(), "Syncronization");
    checkCudaErr(cudaGetLastError(), "GPU");
    for (int i = 0; i < 10; i++) {
		printf("%f -- ", v1[i]);
    printf("%f || ", v2[i]*i);
	}
    for(int i =1; i<10; i++){
        v9[i] = v9[i] + v9[i-1];
        if(v10[i]!=v9[i]){
            printf("erro na execução:");
            break;
        }
    }   
}

'0.000000 -- 0.000000 || 1.000000 -- 1.000000 || 4.000000 -- 4.000000 || 9.000000 -- 9.000000 || 16.000000 -- 16.000000 || 25.000000 -- 25.000000 || 36.000000 -- 36.000000 || 49.000000 -- 49.000000 || 64.000000 -- 64.000000 || 81.000000 -- 81.000000 || erro na execução:6.000000-5.000000;;3;;erro na execução:10.000000-7.000000;;4;;erro na execução:15.000000-9.000000;;5;;erro na execução:21.000000-11.000000;;6;;erro na execução:28.000000-13.000000;;7;;erro na execução:36.000000-15.000000;;8;;erro na execução:45.000000-17.000000;;9;;'

In [0]:
r = open(r"drive/Colab Notebooks/data/2003_td_dataset/Fold1/Norm.test.txt");

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
!unzip data.zip

In [0]:
!git clone https://github.com/Haiga/learning2rank.git

In [0]:
!/opt/bin/nvidia-smi
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb 2> /dev/null
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub
!apt-get update
!apt-get install -qq cuda gcc-5 g++-5 -y
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++
!/usr/local/cuda/bin/nvcc --version

In [0]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [0]:
%load_ext nvcc_plugin

In [0]:
!ls

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive

In [11]:
%%cu
#include <stdio.h>
#define N  64
inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
  if (err != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
  }
  return err;
}
__global__ void matrixMulGPU( int * a, int * b, int * c )
{
  /*
   * Build out this kernel.
   */
    int row = threadIdx.y + blockIdx.y * blockDim.y;
    int col = threadIdx.x + blockIdx.x * blockDim.x;
    
    int val = 0;
    if (row < N && col < N) {
      for (int i = 0; i < N; ++i) {
         val += a[row * N + i] * b[i * N + col];
       }
    
      c[row * N + col] = val;
    }
}

void matrixMulCPU( int * a, int * b, int * c )
{
  int val = 0;
for( int row = 0; row < N; ++row )
    for( int col = 0; col < N; ++col )
    {
      val = 0;
      for ( int k = 0; k < N; ++k )
        val += a[row * N + k] * b[k * N + col];
      c[row * N + col] = val;
    }
}
int main()
{
  int *a, *b, *c_cpu, *c_gpu; 
int size = N * N * sizeof (int); 
  cudaMallocManaged (&a, size);
  cudaMallocManaged (&b, size);
  cudaMallocManaged (&c_cpu, size);
  cudaMallocManaged (&c_gpu, size);

  for( int row = 0; row < N; ++row )
    for( int col = 0; col < N; ++col )
    {
      a[row*N + col] = row;
      b[row*N + col] = col+2;
      c_cpu[row*N + col] = 0;
      c_gpu[row*N + col] = 0;
    }

dim3 threads_per_block(32, 32, 1);
dim3 number_of_blocks(N / threads_per_block.x , N / threads_per_block.y , 1);
matrixMulGPU <<< number_of_blocks, threads_per_block >>> ( a, b, c_gpu );
checkCudaErr(cudaDeviceSynchronize(), "Syncronization");
checkCudaErr(cudaGetLastError(), "GPU");

  matrixMulCPU( a, b, c_cpu );

  bool error = false;
  for( int row = 0; row < N && !error; ++row )
    for( int col = 0; col < N && !error; ++col )
      if (c_cpu[row * N + col] != c_gpu[row * N + col])
      {
        printf("FOUND ERROR at c[%d][%d]\n", row, col);
        error = true;
        break;
      }
if (!error)
    printf("Success!\n");
else
    //printf(error);

  cudaFree(a); cudaFree(b);
  cudaFree( c_cpu ); cudaFree( c_gpu );
}

'Success!\n'